In [1]:
import numpy as np # linear algebra
import pandas as pd # csv I/O

IMG_SIZE = 224
MODEL_NAME = 'resnet50'

DATA_DIR= r'D:\LICENTA\processed_data\size_{size1}x{size2}'.format(size1=IMG_SIZE, size2=IMG_SIZE)
FEATURES_PATH = DATA_DIR +  r'\train_biz_{name}_features.csv'.format(name=MODEL_NAME)

In [2]:
train_photos = pd.read_csv('train_photo_to_biz_ids.csv')
train_photo_to_biz = pd.read_csv('train_photo_to_biz_ids.csv', index_col='photo_id')

train_df = pd.read_csv(FEATURES_PATH)

X = train_df['feature_vector'].values
Y = train_df['label'].values

def convert_label_to_array(str_label):
    str_label = str_label[1:-1]
    str_label = str_label.split(',')
    return [int(x) for x in str_label if len(x) > 0]

def convert_feature_to_vector(str_feature):
    str_feature = str_feature[1:-1]
    str_feature = str_feature.split(',')
    return [float(x) for x in str_feature]

Y = np.array([convert_label_to_array(y) for y in train_df['label']])
X = np.array([convert_feature_to_vector(x) for x in train_df['feature_vector']])

In [3]:
print("X_train: ", X.shape)
print("y_train: ", Y.shape)
print("train_df:")
train_df[0:5]

X_train:  (1996, 2048)
y_train:  (1996,)
train_df:


,business,label,feature_vector
0,1000,"(1, 2, 3, 4, 5, 6, 7)","[0.26928094, 0.67035854, 1.505075, 0.63246781,..."
1,1001,"(0, 1, 6, 8)","[0.061924133, 0.47727728, 1.2030253, 0.3298085..."
2,100,"(1, 2, 4, 5, 6, 7)","[0.28768438, 0.31944394, 2.1715095, 0.38561299..."
3,1006,"(1, 2, 4, 5, 6)","[0.44401026, 0.46946144, 1.987806, 0.19735378,..."
4,1010,"(0, 6, 8)","[0.10140131, 0.52415538, 3.7969804, 0.41625538..."


In [4]:
from sklearn import svm, preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.grid_search import GridSearchCV


mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(Y)

random_state = np.random.RandomState(0)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=random_state)

print('X_train: ',X_train.shape)
print('X_test: ', X_test.shape)
print('y_train: ', y_train.shape)
print('y_test: ', y_test.shape)

X_train:  (1596, 2048)
X_test:  (400, 2048)
y_train:  (1596, 9)
y_test:  (400, 9)


c:\users\eduar\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\users\eduar\appdata\local\programs\python\python36\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [22]:
%%time

clf = OneVsRestClassifier(svm.SVC(kernel='rbf'), n_jobs=-1)
X_train = preprocessing.scale(X_train)
clf.fit(X_train, y_train)

X_test = preprocessing.scale(X_test)
y_pred = clf.predict(X_test)


# parameters = {
#     "estimator__C": [1,2,4,8],
#     "estimator__kernel": ["poly","rbf"],
#     "estimator__degree":[1, 2, 3, 4],
# }

# model_tunning = GridSearchCV(clf, param_grid=parameters,
#                              scoring='f1_micro', n_jobs=-1)
# model_tunning.fit(X, y)


Wall time: 37.2 s


In [6]:
clf.score(X_test, y_test)

0.28749999999999998

In [23]:
# print(model_tunning.best_score_)
# print(model_tunning.best_params_)

from sklearn.metrics import f1_score

print('F1 Score: ', f1_score(y_test, y_pred, average='micro'))
print('F1 Individual Score: ', f1_score(y_test, y_pred, average=None))

F1 Score:  0.844945152683
F1 Individual Score:  [ 0.70992366  0.80526316  0.89847716  0.71465296  0.7745098   0.89754098
  0.94464945  0.76785714  0.91020408]


In [11]:
import pickle

with open(r'D:\LICENTA\models\svm_{size1}x{size2}\svm_{name}.pkl'.format(size1=IMG_SIZE, size2=IMG_SIZE, name=MODEL_NAME), 'wb') as f:
    pickle.dump(clf, f)

In [8]:
# testing with outside photos

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import os

# takes around 45 seconds
model = VGG16(weights='imagenet', include_top=False, pooling='max')

TEST_DIR = r'D:\Retro Test Photos'
        
features = []

for path in list(os.listdir(TEST_DIR)):
    img = image.load_img(os.path.join(TEST_DIR, path), target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features.append(model.predict(img_array).reshape(512,))
    
test = np.mean(np.array(features), axis=0)

test = test.reshape(512,)
test = preprocessing.scale(test)
test = test.reshape(1, -1)
prediction = clf.predict(test)
print(prediction)

Using TensorFlow backend.


[[0 1 1 0 0 1 1 0 0]]


c:\users\eduar\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
c:\users\eduar\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
